# VED: preprocessing → regression baseline → residuals

This notebook is **adapted to the real VED file structure**, where the dynamic data are stored across many files named like `VED_*_week.csv`.

What the notebook does:

1. reads *all* `*_week.csv` files from a directory (e.g., `VED_171101_week.csv`);
2. builds a **trip-level table** (aggregates time-series data → 1 row per trip);
3. computes the target **energy_per_km** (by integrating power over time and dividing by distance);
4. trains a regression model (XGBoost) to predict the “normal” trip energy consumption;
5. saves the model and computes `residual = actual − predicted`.

Recommended setup: run this notebook on Kaggle.

Kaggle dataset link: https://www.kaggle.com/datasets/galievilyas/ved-dataset/data

## 0) Imports and configuration

In [1]:
import os
from pathlib import Path
import glob
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
import joblib

RANDOM_STATE = 42

# --------- ПУТИ (поменяй под себя) ----------
DATA_DIR = Path("/kaggle/input/datasets/galievilyas/ved-dataset/VED_DynamicData_Part*/")   # папка где лежат все VED_*_week.csv
WEEK_GLOB = "VED_*_week.csv"

STATIC_PHEV_EV_XLSX = Path("/kaggle/input/datasets/galievilyas/ved-dataset/VED_Static_Data_PHEV_EV.xlsx")
STATIC_ICE_HEV_XLSX = Path("/kaggle/input/datasets/galievilyas/ved-dataset/VED_Static_Data_ICE_HEV.xlsx")

OUT_DIR = Path("outputs")
MODEL_DIR = Path("models")
OUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# --------- КОЛОНКИ VED (в твоём CSV они такие) ----------
COL_DAYNUM = "DayNum"
COL_VEHID = "VehId"
COL_TRIP = "Trip"
COL_TS_MS = "Timestamp(ms)"
COL_SPEED = "Vehicle Speed[km/h]"
COL_FUEL_RATE = "Fuel Rate[L/hr]"  # для ICE/HEV

# опциональные (могут быть NaN)
COL_AC_KW = "Air Conditioning Power[kW]"
COL_AC_W = "Air Conditioning Power[Watts]"
COL_HEATER_W = "Heater Power[Watts]"
COL_HV_I = "HV Battery Current[A]"
COL_HV_V = "HV Battery Voltage[V]"

# ---- ФИЗИЧЕСКИЕ КОНСТАНТЫ / ДОПУЩЕНИЯ ----
GASOLINE_KWH_PER_LITER = 8.9  # грубая конверсия L топлива -> kWh энергии (LHV). Можно уточнить, но для baseline ок.
SPEED_STOP_THRESHOLD = 5.0    # km/h

pd.options.display.float_format = lambda x: f"{x:.4f}"


## 1) Reading multiple week files

We read **all** `VED_*_week.csv` files from `DATA_DIR`. If there are many files and memory is limited, you can process them one by one (as we do below) and accumulate the aggregated results.

In [2]:
def list_week_files(data_dir: Path, pattern: str) -> list[Path]:
    files = sorted([Path(p) for p in glob.glob(str(data_dir / pattern))])
    if not files:
        raise FileNotFoundError(f"No files found: {data_dir}/{pattern}")
    return files

week_files = list_week_files(DATA_DIR, WEEK_GLOB)
print(f"Found {len(week_files)} week files. Example: {week_files[0]}")


Found 54 week files. Example: /kaggle/input/datasets/galievilyas/ved-dataset/VED_DynamicData_Part1/VED_171101_week.csv


## 2) Target: `energy_per_km` from VED signals

In VED, the dynamic files include vehicle speed and (for ICE/HEV) `Fuel Rate[L/hr]`. For EV/PHEV, battery current/voltage may be available.

For a baseline, we compute:

* fuel power: `P_fuel_kW = FuelRate[L/hr] * 8.9 kWh/L` (since L/hr × kWh/L = kWh/hr = kW)
* battery power: `P_batt_kW = (V * I) / 1000`
* HVAC: `AC_kW` or `AC_W/1000`, plus `Heater_W/1000`
* total power: `P_total_kW = P_fuel_kW + P_batt_kW + HVAC`
* trip energy: integrate `P_total_kW` over time
* distance: integrate speed over time
* `energy_per_km = total_energy_kWh / distance_km`

This is an approximation (especially for fuel-based power), but it is sufficient for a coursework baseline!


In [3]:
def compute_instant_power_kW(df: pd.DataFrame) -> np.ndarray:
    # Fuel power (kW)
    if COL_FUEL_RATE in df.columns:
        fuel = df[COL_FUEL_RATE].astype(float).fillna(0.0).to_numpy()
    else:
        fuel = np.zeros(len(df), dtype=float)
    p_fuel = fuel * GASOLINE_KWH_PER_LITER  # (L/hr)*(kWh/L)=kW

    # Battery power (kW)
    if (COL_HV_I in df.columns) and (COL_HV_V in df.columns):
        i = df[COL_HV_I].astype(float).fillna(0.0).to_numpy()
        v = df[COL_HV_V].astype(float).fillna(0.0).to_numpy()
        p_batt = (v * i) / 1000.0
    else:
        p_batt = np.zeros(len(df), dtype=float)

    # HVAC
    ac = np.zeros(len(df), dtype=float)
    if COL_AC_KW in df.columns:
        ac += df[COL_AC_KW].astype(float).fillna(0.0).to_numpy()
    elif COL_AC_W in df.columns:
        ac += df[COL_AC_W].astype(float).fillna(0.0).to_numpy() / 1000.0

    heater = np.zeros(len(df), dtype=float)
    if COL_HEATER_W in df.columns:
        heater += df[COL_HEATER_W].astype(float).fillna(0.0).to_numpy() / 1000.0

    p_total = p_fuel + p_batt + ac + heater
    return p_total

def integrate_trip_energy_distance(trip_df: pd.DataFrame) -> tuple[float, float, float]:
    """Return (energy_kWh, distance_km, duration_min)"""
    trip_df = trip_df.sort_values(COL_TS_MS)
    t = trip_df[COL_TS_MS].astype(float).to_numpy() / 1000.0  # seconds
    if len(t) < 2:
        return 0.0, 0.0, 0.0
    dt = np.diff(t)
    dt = np.clip(dt, 0.0, None)

    speed_kmh = trip_df[COL_SPEED].astype(float).fillna(0.0).to_numpy()
    speed_kms = speed_kmh / 3600.0

    p_kW = compute_instant_power_kW(trip_df)

    # Integrals using left Riemann sum on intervals [i, i+1)
    energy_kWh = float(np.sum(p_kW[:-1] * (dt / 3600.0)))
    distance_km = float(np.sum(speed_kms[:-1] * dt))
    duration_min = float((t[-1] - t[0]) / 60.0)
    return energy_kWh, distance_km, duration_min


## 3) Trip-level feature aggregation (1 row per trip)

We aggregate the following features:

* duration, distance
* `speed_mean` / `speed_var`
* `accel_mean` / `accel_var` / `accel_p95`
* `stop_go_ratio`, `idle_time_min`

You can also add any other available signals (RPM, Load, OAT, etc.)—below I’ll leave an example showing how to extend this easily.

In [4]:
def compute_trip_features(trip_df: pd.DataFrame) -> dict:
    trip_df = trip_df.sort_values(COL_TS_MS)
    t = trip_df[COL_TS_MS].astype(float).to_numpy() / 1000.0
    if len(t) < 2:
        return {}

    dt = np.diff(t)
    dt = np.clip(dt, 0.0, None)
    dt_sum = float(np.sum(dt)) + 1e-12

    speed_kmh = trip_df[COL_SPEED].astype(float).fillna(0.0).to_numpy()
    speed_ms = speed_kmh * (1000.0 / 3600.0)

    # time-weighted mean/var (weights = dt for intervals, align to speed[:-1])
    w = dt / dt_sum
    speed_mean = float(np.sum(speed_kmh[:-1] * w))
    speed_var = float(np.sum(((speed_kmh[:-1] - speed_mean) ** 2) * w))

    # accel over intervals
    accel = np.diff(speed_ms) / (dt + 1e-12)
    accel = np.nan_to_num(accel, nan=0.0, posinf=0.0, neginf=0.0)
    accel_mean = float(np.sum(accel * w))
    accel_var = float(np.sum(((accel - accel_mean) ** 2) * w))
    accel_p95 = float(np.percentile(accel, 95))

    stop_go_ratio = float(np.mean(speed_kmh < SPEED_STOP_THRESHOLD))
    idle_time_min = float(np.sum(dt[speed_kmh[:-1] < 0.1]) / 60.0)

    # trip-level integrals for target
    energy_kWh, distance_km, duration_min = integrate_trip_energy_distance(trip_df)
    if distance_km <= 1e-6:
        energy_per_km = np.nan
    else:
        energy_per_km = energy_kWh / distance_km

    return {
        "duration_min": duration_min,
        "distance_km": distance_km,
        "speed_mean": speed_mean,
        "speed_var": speed_var,
        "accel_mean": accel_mean,
        "accel_var": accel_var,
        "accel_p95": accel_p95,
        "stop_go_ratio": stop_go_ratio,
        "idle_time_min": idle_time_min,
        "energy_kWh": energy_kWh,
        "energy_per_km": energy_per_km,
    }

def aggregate_week_file(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)

    # создаём стабильный trip_id
    df["trip_id"] = df[COL_VEHID].astype(str) + "_" + df[COL_TRIP].astype(str)

    feats = []
    for trip_id, g in df.groupby("trip_id", sort=False):
        f = compute_trip_features(g)
        if not f:
            continue
        f["trip_id"] = trip_id
        f["VehId"] = int(g[COL_VEHID].iloc[0])
        f["Trip"] = int(g[COL_TRIP].iloc[0])
        feats.append(f)

    return pd.DataFrame(feats)

# Прогон на одном файле (проверка)
sample_trips = aggregate_week_file(week_files[0])
sample_trips.head()


,duration_min,distance_km,speed_mean,speed_var,accel_mean,accel_var,accel_p95,stop_go_ratio,idle_time_min,energy_kWh,energy_per_km,trip_id,VehId,Trip
0,2.2383,2.0546,55.0759,273.2273,-0.0062,0.8663,1.7650,0.0584,0.0000,0.0000,0.0000,8_706,8,706
1,3.9917,2.8922,43.4735,672.6869,0.0220,3.0715,3.7731,0.1470,0.6017,0.0000,0.0000,8_707,8,707
2,1.7700,0.8968,30.3990,391.4863,-0.0501,1.4627,1.7220,0.1008,0.3750,-0.0632,-0.0704,10_1558,10,1558
3,3.8583,3.2690,50.8352,85.0810,0.0016,0.8729,1.3222,0.0000,0.0000,-0.0171,-0.0052,11_1485,11,1485
4,8.1067,6.3116,46.7144,741.2270,0.0069,0.9453,2.0910,0.0600,1.0017,0.0000,0.0000,124_755,124,755


## 4) Building the trip table from all week files

If there are many files, this may take some time. The upside is that you’ll keep a compact trip-level table in RAM, rather than the full time-series data.

In [5]:
trip_tables = []
for i, f in enumerate(week_files):
    tdf = aggregate_week_file(f)
    trip_tables.append(tdf)
    if (i+1) % 5 == 0:
        print(f"Processed {i+1}/{len(week_files)} files. trips so far: {sum(len(x) for x in trip_tables)}")

trips_df = pd.concat(trip_tables, ignore_index=True)

# чистим NaN по таргету/дистанции
trips_df = trips_df.dropna(subset=["energy_per_km"])
trips_df = trips_df[trips_df["distance_km"] > 0.1].reset_index(drop=True)

print("Trips:", trips_df.shape)
trips_df.describe(include="all").T.head(15)


Processed 5/54 files. trips so far: 4172
Processed 10/54 files. trips so far: 7715
Processed 15/54 files. trips so far: 11284
Processed 20/54 files. trips so far: 14144
Processed 25/54 files. trips so far: 17367
Processed 30/54 files. trips so far: 20413
Processed 35/54 files. trips so far: 23322
Processed 40/54 files. trips so far: 26008
Processed 45/54 files. trips so far: 28293
Processed 50/54 files. trips so far: 30758
Trips: (32512, 14)


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
duration_min,32512.0000,NaN,NaN,NaN,8.9442,8.4151,0.1550,3.5983,6.8817,11.4667,168.6817
distance_km,32512.0000,NaN,NaN,NaN,5.4583,6.4761,0.1003,2.1223,4.2541,6.8474,224.4132
speed_mean,32512.0000,NaN,NaN,NaN,38.6670,14.8794,0.3198,29.1719,36.6592,45.7210,115.9534
speed_var,32512.0000,NaN,NaN,NaN,553.7276,320.3532,0.0000,351.2364,521.3446,681.8790,3019.8586
accel_mean,32512.0000,NaN,NaN,NaN,-0.0030,0.0338,-0.2976,-0.0141,-0.0014,0.0088,0.4041
accel_var,32512.0000,NaN,NaN,NaN,1.6189,0.9294,0.0000,1.0137,1.4435,1.9970,17.6734
accel_p95,32512.0000,NaN,NaN,NaN,2.0597,0.8500,0.0000,1.3889,1.8519,2.7778,13.0556
stop_go_ratio,32512.0000,NaN,NaN,NaN,0.1301,0.1098,0.0000,0.0471,0.1077,0.1900,0.9556
idle_time_min,32512.0000,NaN,NaN,NaN,2.0247,3.6793,0.0000,0.3400,1.1267,2.4350,161.3533
energy_kWh,32512.0000,NaN,NaN,NaN,-0.0359,0.7795,-28.7004,0.0000,0.0000,0.0000,71.3665


## 5) Merging static tables (weight/type/class)

The static data are stored in two XLSX files. We merge them by `VehId` and add `Generalized_Weight` as an important static feature.

In [11]:
def load_static_tables(phev_ev_path: Path, ice_hev_path: Path) -> pd.DataFrame:
    phev = pd.read_excel(phev_ev_path)
    ice = pd.read_excel(ice_hev_path)

    # унифицируем колонку типа двигателя
    phev = phev.rename(columns={"EngineType": "VehicleType"})
    ice = ice.rename(columns={"Vehicle Type": "VehicleType"})

    static = pd.concat([phev, ice], ignore_index=True)
    # dedupe in case of overlaps
    static = static.drop_duplicates(subset=["VehId"]).reset_index(drop=True)
    return static

static_df = load_static_tables(STATIC_PHEV_EV_XLSX, STATIC_ICE_HEV_XLSX)
trips_df = trips_df.merge(static_df, on="VehId", how="left")
trips_df["Generalized_Weight"] = pd.to_numeric(trips_df["Generalized_Weight"], errors="coerce")

trips_df[["VehId","VehicleType","Vehicle Class","Generalized_Weight"]].head()


,VehId,VehicleType,Vehicle Class,Generalized_Weight
0,8,ICE,Car,2500.0000
1,8,ICE,Car,2500.0000
2,10,EV,Car,3500.0000
3,11,PHEV,Car,4000.0000
4,124,HEV,NO DATA,3000.0000


## 6) Train/Val/Test split by `VehId` (no leakage)

To prevent the model from memorizing a specific vehicle, we split the data by `VehId`.

In [12]:
def split_by_vehicle(trips: pd.DataFrame, test_size=0.2, val_size=0.2, seed=42):
    vehicles = trips["VehId"].unique()
    train_val, test = train_test_split(vehicles, test_size=test_size, random_state=seed)
    train, val = train_test_split(train_val, test_size=val_size/(1-test_size), random_state=seed)
    return (
        trips["VehId"].isin(train),
        trips["VehId"].isin(val),
        trips["VehId"].isin(test),
    )

train_mask, val_mask, test_mask = split_by_vehicle(trips_df, seed=RANDOM_STATE)
train_df, val_df, test_df = trips_df[train_mask], trips_df[val_mask], trips_df[test_mask]
len(train_df), len(val_df), len(test_df)


(20080, 6137, 6295)

## 7) Training an XGBoost regressor + saving the model

We use numerical features, and optionally one-hot encode categorical features (e.g., `VehicleType`, `Class`, `Drive Wheels`). Here we apply one-hot encoding via `pandas.get_dummies`.

Result: we save the trained model and the `residuals.parquet` table.

In [15]:
TARGET = "energy_per_km"
ID_COLS = ["trip_id", "VehId", "Trip"]

# выберем базовый набор численных
base_num_features = [
    "duration_min","distance_km","speed_mean","speed_var",
    "accel_mean","accel_var","accel_p95",
    "stop_go_ratio","idle_time_min",
    "Generalized_Weight",
]

# категориальные
cat_features = ["VehicleType","Vehicle Class","Transmission","Drive Wheels"]

def make_design(df: pd.DataFrame) -> pd.DataFrame:
    X = df[base_num_features + cat_features].copy()
    X = pd.get_dummies(X, columns=cat_features, dummy_na=True)
    return X

X_train = make_design(train_df)
X_val   = make_design(val_df)
X_test  = make_design(test_df)

# общий список колонок (union)
all_cols = sorted(set(X_train.columns) | set(X_val.columns) | set(X_test.columns))

# одинаковые колонки + одинаковый порядок
X_train = X_train.reindex(columns=all_cols, fill_value=0.0)
X_val   = X_val.reindex(columns=all_cols, fill_value=0.0)
X_test  = X_test.reindex(columns=all_cols, fill_value=0.0)

print(X_train.shape, X_val.shape, X_test.shape)  # должны совпасть по числу фич

model = XGBRegressor(
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.0,
    reg_lambda=1.0,
    random_state=RANDOM_STATE,
    tree_method="hist",
    n_jobs=-1,
    early_stopping_rounds=50,
    eval_metric="rmse"
)

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=50
)

def eval_split(name, X, y):
    pred = model.predict(X)
    mae = mean_absolute_error(y, pred)
    rmse = np.sqrt(mean_squared_error(y, pred))
    print(f"{name}: MAE={mae:.4f}, RMSE={rmse:.4f}")
    return pred

_ = eval_split("VAL", X_val, y_val)
_ = eval_split("TEST", X_test, y_test)

# save model + feature columns
model_path = MODEL_DIR / "xgb_energy_regressor.joblib"
joblib.dump({"model": model, "feature_columns": list(X_train.columns)}, model_path)
print("Saved:", model_path)


(20080, 41) (6137, 41) (6295, 41)
[0]	validation_0-rmse:0.06676
[50]	validation_0-rmse:0.06729
[62]	validation_0-rmse:0.06801
VAL: MAE=0.0171, RMSE=0.0634
TEST: MAE=0.0233, RMSE=0.0815
Saved: models/xgb_energy_regressor.joblib


## 8) Residuals table (`actual`, `predicted`, `residual`)

In [16]:
# predict for all trips
X_all = make_design(trips_df)
X_all = X_all.reindex(columns=list(X_train.columns), fill_value=0.0)

payload = joblib.load(model_path)
mdl = payload["model"]

pred_all = mdl.predict(X_all)
res_df = trips_df[ID_COLS + [TARGET]].copy()
res_df["predicted_energy_per_km"] = pred_all
res_df["residual"] = res_df[TARGET] - res_df["predicted_energy_per_km"]

out_path = OUT_DIR / "residuals.parquet"
res_df.to_parquet(out_path, index=False)
print("Saved residuals:", out_path)

res_df.head()


Saved residuals: outputs/residuals.parquet


,trip_id,VehId,Trip,energy_per_km,predicted_energy_per_km,residual
0,8_706,8,706,0.0000,-0.0067,0.0067
1,8_707,8,707,0.0000,-0.0067,0.0067
2,10_1558,10,1558,-0.0704,-0.0399,-0.0305
3,11_1485,11,1485,-0.0052,-0.0482,0.0429
4,124_755,124,755,0.0000,-0.0029,0.0029


## 9) Saving model

In [17]:
import joblib
import numpy as np
from pathlib import Path

ARTIFACT_DIR = Path("models")
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

feature_columns = list(X_train.columns)

artifact = {
    "model": model,
    "feature_columns": feature_columns,
    # чтобы потом гарантированно делать fillna одинаково
    "feature_medians": X_train.median(numeric_only=True).to_dict(),
    # background для LIME (не обязательно, но удобно)
    "lime_background": X_train.sample(n=min(5000, len(X_train)), random_state=RANDOM_STATE).astype(float),
}

joblib.dump(artifact, ARTIFACT_DIR / "xgb_energy_artifact.joblib")
print("Saved:", ARTIFACT_DIR / "xgb_energy_artifact.joblib")


Saved: models/xgb_energy_artifact.joblib


## 10) Load the model

In [18]:
import joblib
import pandas as pd

artifact = joblib.load("models/xgb_energy_artifact.joblib")
model = artifact["model"]
feature_columns = artifact["feature_columns"]
feature_medians = artifact["feature_medians"]

def predict_from_raw_design(X_design: pd.DataFrame) -> pd.Series:
    # 1) привести к тем же колонкам и порядку
    X = X_design.reindex(columns=feature_columns, fill_value=0.0)
    # 2) fillna теми же медианами
    for c, m in feature_medians.items():
        if c in X.columns:
            X[c] = X[c].fillna(m)
    return pd.Series(model.predict(X), index=X_design.index)
